In [449]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [450]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB
import pandas as pd

db = DB()

In [451]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz', "test"]

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp",
    "test": "22skzmqfdWrjJylampe0kt"
})

In [452]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['jazz', 'pop'],
 ['jazz'],
 ['k-pop'],
 ['ballad', 'k-pop', 'jazz'],
 ['jazz', 'k-pop', 'hip-hop'],
 ['pop'],
 ['jazz', 'hip-hop', 'test'],
 ['hip-hop', 'pop'],
 ['test'],
 ['k-pop', 'ballad']]

In [453]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623f2e6cff1c441a392ab4f9', '623f2e6cff1c441a392ab4fa', '623f2e6cff1c441a392ab4fb', '623f2e6cff1c441a392ab4fc', '623f2e6cff1c441a392ab4fd', '623f2e6cff1c441a392ab4fe', '623f2e6dff1c441a392ab4ff', '623f2e6dff1c441a392ab500', '623f2e6dff1c441a392ab501', '623f2e6dff1c441a392ab502']


,id,name,artists,artists_name
0,7tIOwGFRTM08JdAeKOELGp,Blue room,3rxeQlsv0Sc2nyYaZ5W71T,Chet Baker
1,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan


In [454]:
features_list = list()
genre_seed_list = list()
for sel_track in sel_track_list:
    genres = spotify.get_genres(token, sel_track)
    genre_seed_list.append(genres)
    
    features = spotify.get_features(token, sel_track)
    features_list.append(features)
    
    db.observe_seed_zone(features)
    
print(genre_seed_list[9])
features_list[9]

                  artists     genres
0  6YVMFz59CuY7ngCxTxjpxE  k-pop,pop
1  02SENF0XVfxRTZrniEV2aF  k-pop,pop


,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3dbLT62Cvs46Ju7a8gpr36,0.727,0.879,9,-1.167,0.1290,0.124,0.000012,0.244,0.671,146.959
1,3UPjb91Fwm7u2tAm92Bk0p,0.554,0.526,7,-5.210,0.0394,0.808,0.000000,0.074,0.230,129.845


In [455]:
reco_list = list()
reco_features_list = list()

for idx, sel_track in enumerate(sel_track_list):
    print(idx)
    features = features_list[idx]
    genres = genre_seed_list[idx]
    
    recos = spotify.get_recommend(sel_track, features, genres, token)
    reco_features = spotify.get_features(token, recos)
    
    reco_list.append(recos)
    reco_features_list.append(reco_features)

0
1
2
3
4
5
6
7
8
9


In [456]:
reco_list[0].head()

,id,name,artists,artists_name
0,4CZfPaDW5madfScpZl0nDU,Ahmad's Blues,6gc6oo3u2f7SqTd4mhe81O,Ahmad Jamal
1,7xCnic5Ywu9RkzsoMGZyoZ,My Funny Valentine,0LyfQWJT6nXafLPZqxe9Of,Various Artists
2,4d3XHYFFuYYzxWr2cJ6yQl,Angel Eyes - Studio,6dUZplQfg5blo0h9HiJ94d,Tommy Flanagan
3,7uAJuE1qVnXD3zasCqnhOu,Miss D. D.,6hrEc1XqVcGLK0Di6zVADR,Mary Lou Williams
4,3jqbNKYAxTIYGO22RzhK4K,Over The Rainbow,3kUKwTJdH8FuWzF8p6Dg9E,Dave Brubeck


In [457]:
reco_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,4CZfPaDW5madfScpZl0nDU,0.508,0.1500,5,-20.844,0.0418,0.990,0.899,0.144,0.252,173.360
1,7xCnic5Ywu9RkzsoMGZyoZ,0.374,0.0676,5,-18.741,0.0416,0.974,0.897,0.129,0.235,145.849
2,4d3XHYFFuYYzxWr2cJ6yQl,0.487,0.1450,5,-18.670,0.0522,0.876,0.822,0.149,0.142,110.134
3,7uAJuE1qVnXD3zasCqnhOu,0.489,0.0566,5,-24.377,0.0405,0.911,0.938,0.109,0.603,137.998
4,3jqbNKYAxTIYGO22RzhK4K,0.615,0.0623,5,-21.490,0.0667,0.991,0.932,0.073,0.104,131.364


In [458]:
norm_features_list = list()

for idx, features in enumerate(features_list):
    reco_features = reco_features_list[idx]
    norm_features = dp.make_norm(features, reco_features)
    norm_features_list.append(norm_features)
    
norm_features_list[0].head()

,id,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,7tIOwGFRTM08JdAeKOELGp,0.597080,0.039749,0.454545,0.283761,0.100272,0.993968,0.952138,0.113573,0.343145,0.477166
1,3gzIK2dGSVvPBiohsTwEWZ,0.607299,0.631358,0.000000,0.840775,0.067641,0.058125,0.000172,0.107056,0.623417,0.638131
2,4CZfPaDW5madfScpZl0nDU,0.416058,0.164708,0.454545,0.273421,0.044188,0.994973,0.915479,0.165716,0.229521,0.864559
3,7xCnic5Ywu9RkzsoMGZyoZ,0.220438,0.072938,0.454545,0.361101,0.043508,0.978888,0.913442,0.141274,0.211124,0.659602
4,4d3XHYFFuYYzxWr2cJ6yQl,0.385401,0.159140,0.454545,0.364061,0.079538,0.880368,0.837067,0.173863,0.110486,0.393524


In [459]:
kmeans_list = list()

for idx, norm_features in enumerate(norm_features_list):
    print("-- 라운드 {}".format(idx + 1))
    kmeans = utils.KMeans(
        datas=norm_features
    )
    kmeans.run(early_stop_cnt=3)
    clear_output(wait=True)
    
    kmeans_list.append(kmeans)

-- 라운드 10
1 round : ECV 28.78%
2 round : ECV 61.33%
3 round : ECV 64.21%
4 round : ECV 64.91%
5 round : ECV 65.17%
6 round : ECV 65.35%
7 round : ECV 65.47%
8 round : ECV 65.53%
9 round : ECV 65.73%
10 round : ECV 65.84%
11 round : ECV 65.85%
12 round : ECV 65.86%
13 round : ECV 65.86%
14 round : ECV 65.86%
15 round : ECV 65.86%
16 round : ECV 65.86%
Clustering End.


In [460]:
filterings = list()
recos_list = list()

for idx, kmeans in enumerate(kmeans_list):
    sel_track = sel_track_list[idx]
    recos = reco_list[idx]
    
    _filtering_music_list = dp.music_filtering(sel_track, kmeans)
    filterings.append(_filtering_music_list)
    
    reco_musics = [_ in _filtering_music_list for _ in recos['id']]
    recos_list.append(recos[reco_musics])
    
recos_list[0].head()

,id,name,artists,artists_name
0,4CZfPaDW5madfScpZl0nDU,Ahmad's Blues,6gc6oo3u2f7SqTd4mhe81O,Ahmad Jamal
1,7xCnic5Ywu9RkzsoMGZyoZ,My Funny Valentine,0LyfQWJT6nXafLPZqxe9Of,Various Artists
2,4d3XHYFFuYYzxWr2cJ6yQl,Angel Eyes - Studio,6dUZplQfg5blo0h9HiJ94d,Tommy Flanagan
3,7uAJuE1qVnXD3zasCqnhOu,Miss D. D.,6hrEc1XqVcGLK0Di6zVADR,Mary Lou Williams
4,3jqbNKYAxTIYGO22RzhK4K,Over The Rainbow,3kUKwTJdH8FuWzF8p6Dg9E,Dave Brubeck


In [461]:
for idx,_mail in enumerate(recos_list):
    mail_box_id = mail_boxes_list[idx]
    
    res = db.regist_mail(mail_box_id, _mail)
    mail_id = res.inserted_id
    
    print("우체통({})에\n추천 플레이리스트 MuLetter({})가 등록되었습니다.".format(
        mail_box_id
        ,str(mail_id)))

우체통(623f2e6cff1c441a392ab4f9)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab503)가 등록되었습니다.
우체통(623f2e6cff1c441a392ab4fa)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab504)가 등록되었습니다.
우체통(623f2e6cff1c441a392ab4fb)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab505)가 등록되었습니다.
우체통(623f2e6cff1c441a392ab4fc)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab506)가 등록되었습니다.
우체통(623f2e6cff1c441a392ab4fd)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab507)가 등록되었습니다.
우체통(623f2e6cff1c441a392ab4fe)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab508)가 등록되었습니다.
우체통(623f2e6dff1c441a392ab4ff)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab509)가 등록되었습니다.
우체통(623f2e6dff1c441a392ab500)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab50a)가 등록되었습니다.
우체통(623f2e6dff1c441a392ab501)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab50b)가 등록되었습니다.
우체통(623f2e6dff1c441a392ab502)에
추천 플레이리스트 MuLetter(623f2e82ff1c441a392ab50c)가 등록되었습니다.


### reply

In [462]:
import math as mt

In [463]:
track_list = list()
reco_track_list = list()

for mail_box_id in mail_boxes_list:
    tracks = db.get_mail_box(mail_box_id)
    reco_tracks = db.get_mail(mail_box_id)

    track_list.append(tracks)
    reco_track_list.append(reco_tracks)
    
track_list[0].head()

,id,name,artists,artists_name
0,7tIOwGFRTM08JdAeKOELGp,Blue room,3rxeQlsv0Sc2nyYaZ5W71T,Chet Baker
1,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan


In [464]:
sel_track_list = list()

for idx, reco_tracks in enumerate(reco_track_list):
    mail_box_id = mail_boxes_list[idx]
    n = mt.floor(len(reco_tracks) * 0.075)
    n = 2 if n <= 1 else n
    
    sel_tracks = reco_tracks.sample(n=n)
    db.new_mail_box_seed(mail_box_id, sel_tracks)

    sel_track_list.append(sel_tracks)

In [465]:
# 1. Get Token
token = spotify.get_token()

_sel_track_list = list()
features_list = list()
recos_list = list()
reco_features_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print(idx)
    tracks = track_list[idx]
    _sel_tracks = pd.concat([tracks,sel_tracks], ignore_index=True)
    genres = spotify.get_genres(token, _sel_tracks)

    og_features = spotify.get_features(token, tracks)
    reply_features = spotify.get_features(token, sel_tracks)
    db.observe_seed_zone(reply_features)

    recos = spotify.get_recommend(sel_tracks, reply_features, genres, token,
                                 og=tracks)
    
    if str(type(recos)) == "<class 'requests.models.Response'>":
        print(recos.url)
        print(idx,recos.json())
        
    reco_features = spotify.get_features(token, recos)
    features = pd.concat([og_features, reply_features], ignore_index=True)
    
    features_list.append(features)
    recos_list.append(recos)
    reco_features_list.append(reco_features)
    _sel_track_list.append(_sel_tracks)

0
1
2
3
4
5
6
7
8
9


In [466]:
norm_features_list = list()

for idx, reco_features in enumerate(reco_features_list):
    features = features_list[idx]
    norm_features = dp.make_norm(features, reco_features)

    norm_features_list.append(norm_features)

In [467]:
filterings = list()
kmeans_list = list()

for idx, sel_tracks in enumerate(sel_track_list):
    print("{}번 박스".format(idx))
    norm_features = norm_features_list[idx]
    
    _1, _2 = utils.reco_KMeans(norm_features, sel_tracks)
    
    filterings.append(_1)
    kmeans_list.append(_2)

1 round : ECV 19.65%
2 round : ECV 46.55%
3 round : ECV 50.64%
4 round : ECV 51.74%
5 round : ECV 52.11%
6 round : ECV 52.18%
7 round : ECV 52.23%
8 round : ECV 52.26%
9 round : ECV 52.26%
10 round : ECV 52.26%
11 round : ECV 52.26%
12 round : ECV 52.26%
13 round : ECV 52.26%
14 round : ECV 52.26%
Clustering End.


In [468]:
reco_musics = [_ in filterings[1] for _ in recos_list[1]['id']]
recos_list[1][reco_musics]

,id,name,artists,artists_name
0,1huVzhJw5mIQDTk0zLjEmu,I Ain't Got Nobody,2mY5u4CceAPrpBnse1WpFr,Earl Hines
2,7vk7u7kaqw5A6BtkfoxQf3,Do You?,0fauHpmSHwodVYIjTqOGHz,Yiruma
4,17LII4B6V1AhONoueVwoJU,A Blue Time,6dUZplQfg5blo0h9HiJ94d,Tommy Flanagan
5,2hOd88IAX1JT3EfktlSp4D,"Here, There and Everywhere","7uUBTiZ2u5b40vymlFmXrn,0NDc5hTBeKzBrfUnVSdUdg","George Shearing,Andrew Simpkins"
10,5eFFsShr8vchESCk8HGxvn,"Goodbye, Old Girl",4L9xEztn5PKQIO5WnI5W3u,Art Farmer
17,2OqRnfC8QqRdcUfVGtidcr,When Your Lover Has Gone - Instrumental,"4L9xEztn5PKQIO5WnI5W3u,3ZUZYvTkSr7kJQyAXVpqaL","Art Farmer,Donald Byrd"
24,0vwTHK0RI01avuLIrAHp8C,It Ain’t Necessarily So,6hrEc1XqVcGLK0Di6zVADR,Mary Lou Williams
34,0MLmreUxibaRCpjvEHIkEg,"4 Impromptus, Op. 142, D. 935: Impromptu No. 4...","2p0UyoPfYfI76PCStuXfOP,5Kb0Qf13EyYtVJvzCdI9M7","Franz Schubert,Jenő Jandó"
36,1N8kgdsicbNkQE0QettJvg,Birth,0F3Aew9DSd6fb6192K1K0Y,Keith Jarrett
45,1hqQD9G2qUMq3iiDhqhetT,Exactly Like You,0ldU0QJm31y0d6f57R1G2A,Oscar Peterson Trio


In [469]:
for idx,mail_box_id in enumerate(mail_boxes_list):
    recos = recos_list[idx]
    filtering = filterings[idx]
    reco_musics = [_ in filtering for _ in recos['id']]
    
    db.new_mail(mail_box_id,
               recos[reco_musics])